In [2]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from city_sustainability.models import unet_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from city_sustainability.quality import life_quality
from tensorflow.keras.applications.vgg16 import VGG16
from city_sustainability.preprocessing import image_resize
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from city_sustainability.loading_data_448x448 import loading_paths_448
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from city_sustainability.loading_data_448x448 import image_and_label_arrays_448
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, UpSampling2D, Concatenate, Input

2023-05-28 15:48:32.870234: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 15:48:32.893346: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-28 15:48:32.894245: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 15:48:33.307720: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
model_full = unet_model.build_vgg16_model_full(input_shape=(448, 448, 3), num_classes = 9)

2023-05-28 12:03:09.994671: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-28 12:03:09.996861: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Retrieve image_paths and label_paths
data_path_1 ='../../raw_data/OpenEarthMap_wo_xBD'
image_paths_448, label_paths_448 = loading_paths_448(data_path_1)

In [5]:
unet_model.compile_model(model_full)

In [6]:
model_full.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 448, 448, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 448, 448, 64  36928       ['block1_conv1[0][0]']           
                                )                                                             

                                                                                                  
 batch_normalization_2 (BatchNo  (None, 112, 112, 25  1024       ['conv2d_2[0][0]']               
 rmalization)                   6)                                                                
                                                                                                  
 dropout_2 (Dropout)            (None, 112, 112, 25  0           ['batch_normalization_2[0][0]']  
                                6)                                                                
                                                                                                  
 up_sampling2d_3 (UpSampling2D)  (None, 224, 224, 25  0          ['dropout_2[0][0]']              
                                6)                                                                
                                                                                                  
 concatena

In [11]:
# Retrieve image_paths and label_paths
data_path_1 ='../../raw_data/OpenEarthMap_wo_xBD'
image_paths_448, label_paths_448 = loading_paths_448(data_path_1)

# Define batch size and sampling ratio
batch_size = 64
sampling_ratio = 0.1

# Calculate the number of batches
num_batches = int(np.ceil(len(image_paths_448) * sampling_ratio / batch_size))

# Iterate over batches
for batch_index in range(num_batches):
    # Calculate the start and end indices for the current batch
    start_index = int(batch_index * batch_size / sampling_ratio)
    end_index = int((batch_index + 1) * batch_size / sampling_ratio)

    # Load a batch of image and label arrays
    X_batch, y_batch = image_and_label_arrays_448(image_paths_448[start_index:end_index], label_paths_448[start_index:end_index], sampling_ratio=sampling_ratio)

    # Split the batch into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch, test_size=0.3)

    # Scale the batch
    X_train_scaled = X_train / 255
    X_test_scaled = X_test / 255
    
    # Train the model on the current batch
    unet_model.train_model(model_full, X_train_scaled, y_train,epochs=50, batch_size=batch_size, validation_split=0.2)

Epoch 1/50


2023-05-28 12:05:04.410238: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5394923520 exceeds 10% of free system memory.
2023-05-28 12:05:08.870867: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5394923520 exceeds 10% of free system memory.


1/1 [==============================] - 52s 52s/step - loss: 2.8241 - accuracy: 0.1291 - compute_iou: 0.0613 - val_loss: 7.9969 - val_accuracy: 0.0223 - val_compute_iou: 0.0161 - lr: 1.0000e-04
Epoch 2/50


2023-05-28 12:05:56.008367: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5394923520 exceeds 10% of free system memory.
2023-05-28 12:06:00.797232: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5394923520 exceeds 10% of free system memory.


1/1 [==============================] - 52s 52s/step - loss: 2.6339 - accuracy: 0.1626 - compute_iou: 0.0701 - val_loss: 6.7521 - val_accuracy: 0.0539 - val_compute_iou: 0.0333 - lr: 1.0000e-04
Epoch 3/50


2023-05-28 12:06:47.900452: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 5394923520 exceeds 10% of free system memory.


1/1 [==============================] - 51s 51s/step - loss: 2.4708 - accuracy: 0.2004 - compute_iou: 0.0801 - val_loss: 5.7900 - val_accuracy: 0.1129 - val_compute_iou: 0.0650 - lr: 1.0000e-04
Epoch 4/50
1/1 [==============================] - 51s 51s/step - loss: 2.3302 - accuracy: 0.2379 - compute_iou: 0.0917 - val_loss: 5.0956 - val_accuracy: 0.1875 - val_compute_iou: 0.1070 - lr: 1.0000e-04
Epoch 5/50
1/1 [==============================] - 50s 50s/step - loss: 2.2044 - accuracy: 0.2736 - compute_iou: 0.1050 - val_loss: 4.6353 - val_accuracy: 0.2596 - val_compute_iou: 0.1512 - lr: 1.0000e-04
Epoch 6/50
1/1 [==============================] - 51s 51s/step - loss: 2.0951 - accuracy: 0.3062 - compute_iou: 0.1196 - val_loss: 4.3619 - val_accuracy: 0.3145 - val_compute_iou: 0.1906 - lr: 1.0000e-04
Epoch 7/50
1/1 [==============================] - 51s 51s/step - loss: 2.0022 - accuracy: 0.3348 - compute_iou: 0.1343 - val_loss: 4.2246 - val_accuracy: 0.3526 - val_compute_iou: 0.2224 - lr: 1.

1/1 [==============================] - 50s 50s/step - loss: 1.0376 - accuracy: 0.6459 - compute_iou: 0.3462 - val_loss: 1.8560 - val_accuracy: 0.4274 - val_compute_iou: 0.2478 - lr: 1.0000e-05
Epoch 32/50
1/1 [==============================] - 51s 51s/step - loss: 1.0356 - accuracy: 0.6465 - compute_iou: 0.3467 - val_loss: 1.8338 - val_accuracy: 0.4297 - val_compute_iou: 0.2488 - lr: 1.0000e-05
Epoch 33/50
1/1 [==============================] - 50s 50s/step - loss: 1.0331 - accuracy: 0.6473 - compute_iou: 0.3474 - val_loss: 1.8125 - val_accuracy: 0.4319 - val_compute_iou: 0.2497 - lr: 1.0000e-05
Epoch 34/50
1/1 [==============================] - 50s 50s/step - loss: 1.0317 - accuracy: 0.6477 - compute_iou: 0.3478 - val_loss: 1.7921 - val_accuracy: 0.4341 - val_compute_iou: 0.2506 - lr: 1.0000e-05
Epoch 35/50
1/1 [==============================] - 50s 50s/step - loss: 1.0294 - accuracy: 0.6486 - compute_iou: 0.3485 - val_loss: 1.7727 - val_accuracy: 0.4361 - val_compute_iou: 0.2515 - lr

In [13]:
#Save Model Full
model_full.save('../../model_training/marcel')

2023-05-28 14:31:34.294959: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-28 14:31:34.321090: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-28 14:31:34.345701: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-05-28

INFO:tensorflow:Assets written to: ../../model_training/marcel/assets


INFO:tensorflow:Assets written to: ../../model_training/marcel/assets


In [15]:
#Evaluate Model Full
unet_model.evaluate_model(model_full, X_test_scaled, y_test)

1/1 [==============================] - 2s 2s/step
Evaluation results:
Loss: 1.9868
Accuracy: 0.4512
Model IoU: 0.3007
Calculated IoU: 0.3007


1.9868457317352295

In [ ]:
# Retrieve image_paths and label_paths
data_path_1 = '../../raw_data/OpenEarthMap_wo_xBD'
image_paths_448, label_paths_448 = loading_paths_448(data_path_1)
# Import all image array as X and label array as y
X, y = image_and_label_arrays_448(image_paths_448, label_paths_448, sampling_ratio=1)
# Apply holdout
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
# Scale X and y
X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

In [ ]:
# Create the model
model_full = unet_model.build_vgg16_model_full(input_shape=(448, 448, 3), num_classes = 9)

In [ ]:
# Compile the model
unet_model.compile_model(model_full)

In [ ]:
model_full.summary()

In [ ]:
# This line runs the model without batches.
unet_model.train_model(model_full, X_train_scaled, y_train, epochs = 1, batch_size=18, validation_split=0.2)

In [1]:
# Retrieve image_paths and label_paths
data_path_1 ='../../raw_data/OpenEarthMap_wo_xBD'
image_paths_448, label_paths_448 = loading_paths_448(data_path_1)

# Define batch size and sampling ratio
batch_size = 64
sampling_ratio = 0.1

# Calculate the number of batches
num_batches = int(np.ceil(len(image_paths_448) * sampling_ratio / batch_size))

# Iterate over batches
for batch_index in range(num_batches):
    # Calculate the start and end indices for the current batch
    start_index = int(batch_index * batch_size / sampling_ratio)
    end_index = int((batch_index + 1) * batch_size / sampling_ratio)

    # Load a batch of image and label arrays
    X_batch, y_batch = image_and_label_arrays_448(image_paths_448[start_index:end_index], label_paths_448[start_index:end_index], sampling_ratio=sampling_ratio)

    # Split the batch into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_batch, y_batch, test_size=0.3)

    # Scale the batch
    X_train_scaled = X_train / 255
    X_test_scaled = X_test / 255
    
    # Train the model on the current batch
    unet_model.train_model(model_full, X_train_scaled, y_train,epochs=50, batch_size=batch_size, validation_split=0.2)

NameError: name 'loading_paths_448' is not defined

In [ ]:
# # This code below runs the model with batches
# # Create an instance of ImageDataGenerator
# datagen = ImageDataGenerator(validation_split=0.2)

# # Generate train and valid data iterator
# train_iterator = datagen.flow(X_train_scaled, y_train, subset='training', batch_size=64)
# valid_iterator = datagen.flow(X_train_scaled, y_train, subset='validation', batch_size=64)
# steps_per_epoch = len(train_iterator)

# # Train the model in batches
# epochs = 3
# for epoch in range(epochs):
#     print("Epoch {}/{}".format(epoch + 1, epochs))
#     for step in range(steps_per_epoch):
#         X_batch, y_batch = next(train_iterator)
#         unet_model.train_model(model_full, X_batch, y_batch)
    
#     # Evaluate the model on the validation data
#     steps_valid = len(valid_iterator)
#     total_val_loss = 0
#     for step in range(steps_valid):
#         X_val_batch, y_val_batch = next(valid_iterator)
#         validation_loss = unet_model.evaluate_model(model_full, X_val_batch, y_val_batch)
#         total_val_loss += validation_loss
#     print("Average validation loss: {:.4f}".format(total_val_loss/steps_valid))